In [1]:
from conn.connection import ConnectionManager

In [2]:
from utils import *

In [3]:
con = ConnectionManager()

20:51:29 SMART HOME/CONNECTION - ConnectionManager inicializado.
IP local: 192.168.18.179
IP do dispositivo (TCP): 192.168.18.189
Multicast: 239.1.1.1:7000
Porta TCP: 5006


In [4]:
def callback_fun(msg, add):
    print(msg,add)

ValueError: Não foi possível obter o IP local ou a máscara de rede.

In [24]:
import ipaddress
import subprocess
import platform
from socket import inet_aton, error, AF_INET
from psutil import net_if_addrs
import random

from socket import socket, AF_INET, SOCK_DGRAM
def get_local_ip():
    with socket(AF_INET, SOCK_DGRAM) as s:
        try:
            s.connect(("8.8.8.8", 80))
            return s.getsockname()[0]
        except Exception:
            return "127.0.0.1" 

def get_local_ip_and_subnet_mask():
    local_ip = get_local_ip()
    for _, addrs in net_if_addrs().items():
        for addr in addrs:
            if addr.family == AF_INET and addr.address == local_ip:
                return local_ip, addr.netmask
    return None, None

def is_ip_valid(ip):
    try:
        inet_aton(ip)
        param = "-n" if platform.system().lower() == "windows" else "-c"
        command = ["ping", param, "1", ip]
        return subprocess.call(command, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL) == 0
    except error:
        return False

def list_ips():
    local_ip, subnet_mask = get_local_ip_and_subnet_mask()
    if not local_ip or not subnet_mask:
        raise ValueError("Não foi possível obter o IP local ou a máscara de rede.")
    network = ipaddress.IPv4Network(f"{local_ip}/{subnet_mask}", strict=False)
    hosts = list(network.hosts())
    return [str(host) for host in hosts]

def generate_ip(offset):
    available_ips = list_ips()
    if offset < 0 or offset >= len(available_ips):
        raise ValueError(f"O offset fornecido ({offset}) está fora do intervalo da rede.")
    for i in range(offset, len(available_ips)):
        candidate_ip = available_ips[i]
        if is_ip_valid(candidate_ip):
            return candidate_ip
    raise ValueError("Não foi possível encontrar um IP válido e não utilizado na rede.")


def generate_multicast_group():
    return f"239.{random.randint(0, 255)}.{random.randint(0, 255)}.{random.randint(1, 254)}"


In [31]:
import psutil
import subprocess
import platform
import socket


def is_ip_configured(ip: str) -> bool:
    """
    Verifica se o IP fornecido está configurado em alguma interface de rede local.
    """
    for iface, addrs in psutil.net_if_addrs().items():
        for addr in addrs:
            if addr.family == socket.AF_INET and addr.address == ip:
                return True
    return False


def get_network_interface(subnet: str = "192.168.1.") -> str:
    """
    Retorna a interface de rede associada a uma sub-rede específica.
    """
    for iface, addrs in psutil.net_if_addrs().items():
        for addr in addrs:
            if addr.family == socket.AF_INET and addr.address.startswith(subnet):
                return iface
    raise RuntimeError(f"Nenhuma interface encontrada na sub-rede {subnet}")


def configure_ip(ip: str) -> None:
    """
    Verifica e configura o IP na interface de rede apropriada.
    Funciona para sistemas Ubuntu e Windows.
    """
    if is_ip_configured(ip):
        print(f"O IP {ip} já está configurado.")
        return

    system = platform.system().lower()
    try:
        if system == "linux":
            # Ubuntu ou distribuições baseadas em Linux
            interface = get_network_interface()
            print(f"Interface de rede encontrada: {interface}")
            command = ["sudo", "ip", "addr", "add", f"{ip}/24", "dev", interface]
            subprocess.run(command, check=True)
            print(f"IP {ip} atribuído com sucesso à interface {interface}.")

        elif system == "windows":
            # Configuração para Windows usando netsh
            interface = get_network_interface()
            command = [
                "netsh", "interface", "ip", "add", "address", interface, ip, "255.255.255.0"
            ]
            subprocess.run(command, check=True)
            print(f"IP {ip} atribuído com sucesso à interface {interface}.")

        else:
            raise NotImplementedError(f"Sistema operacional não suportado: {system}")

    except subprocess.CalledProcessError as e:
        print(f"Erro ao atribuir o IP {ip}: {e}")
    except RuntimeError as e:
        print(f"Erro: {e}")
    except NotImplementedError as e:
        print(f"Erro: {e}")


def start_tcp_server(ip: str, port: int):
    """
    Configura um servidor TCP no IP e porta especificados.
    """
    try:
        # Verifica e configura o IP antes de criar o socket
        configure_ip(ip)

        # Inicia o socket TCP
        server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        server_socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
        server_socket.bind((ip, port))
        server_socket.listen(5)
        print(f"[TCP] Servidor escutando em {ip}:{port}")

        while True:
            client_socket, client_address = server_socket.accept()
            print(f"[TCP] Conexão estabelecida com {client_address}")
            data = client_socket.recv(1024)
            if data:
                print(f"[TCP] Dados recebidos: {data}")
            client_socket.close()

    except Exception as e:
        print(f"[TCP] Erro na configuração do servidor TCP: {e}")


if __name__ == "__main__":
    # Configuração do IP e porta
    device_ip = "192.168.1.5"
    device_port = 5006

    # Inicia o servidor TCP
    start_tcp_server(device_ip, device_port)


Interface de rede encontrada: wlp0s20f3
IP 192.168.1.5 atribuído com sucesso à interface wlp0s20f3.
[TCP] Servidor escutando em 192.168.1.5:5006


KeyboardInterrupt: 

In [17]:

def get_local_ip_and_subnet_mask():
    local_ip = get_local_ip()
    for iface, addrs in net_if_addrs().items():
        for addr in addrs:
            print(addr.address, addr.netmask)
            if addr.family == AF_INET:
                # Usa 'addr.address' diretamente para comparar se é o IP local
                if addr.address == local_ip:
                    return addr.address, addr.netmask
    return None, None


In [16]:

def get_local_ip_and_subnet_mask():
    local_ip = get_local_ip()
    for iface, addrs in net_if_addrs().items():
        for addr in addrs:
            if addr.family == AF_INET and addr.address == local_ip:
                return addr.address, addr.netmask
    return None, None

In [26]:
get_local_ip_and_subnet_mask()

('192.168.1.106', '255.255.255.0')

In [23]:

from socket import socket, AF_INET, SOCK_DGRAM
def get_local_ip():
    with socket(AF_INET, SOCK_DGRAM) as s:
        try:
            s.connect(("8.8.8.8", 80))
            return s.getsockname()[0]
        except Exception:
            return "127.0.0.1" 

In [34]:
import ipaddress
import subprocess
import platform
from socket import inet_aton, error, AF_INET, socket, SOCK_DGRAM
from psutil import net_if_addrs
import random
from utils.LoggerConfig import LoggerConfig




logger = LoggerConfig("IPConfig").get_logger()

def get_local_ip():
    with socket(AF_INET, SOCK_DGRAM) as s:
        try:
            s.connect(("8.8.8.8", 80))
            return s.getsockname()[0]
        except Exception:
            return "127.0.0.1" 


def get_local_ip_and_subnet_mask():
    logger.debug("Obtendo IP local e máscara de rede...")
    local_ip = get_local_ip()
    for _, addrs in net_if_addrs().items():
        for addr in addrs:
            if addr.family == AF_INET and addr.address == local_ip:
                logger.debug(f"IP local e máscara de rede obtidos: {local_ip}, {addr.netmask}")
                return local_ip, addr.netmask
    return None, None

def is_ip_valid(ip):
    logger.debug(f"Verificando se o IP {ip} é válido...")
    try:
        inet_aton(ip)
        param = "-n" if platform.system().lower() == "windows" else "-c"
        command = ["ping", param, "1", ip]
        return subprocess.call(command, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL) != 0
    except error:
        return False

def list_ips():
    local_ip, subnet_mask = get_local_ip_and_subnet_mask()
    if not local_ip or not subnet_mask:
        raise ValueError("Não foi possível obter o IP local ou a máscara de rede.")
    network = ipaddress.IPv4Network(f"{local_ip}/{subnet_mask}", strict=False)
    hosts = list(network.hosts())
    return [str(host) for host in hosts]

def generate_ip(offset):
    logger.debug(f"Gerando IP com offset {offset}...")
    available_ips = list_ips()  
    if offset < 0 or offset >= len(available_ips):
        raise ValueError(f"O offset fornecido ({offset}) está fora do intervalo da rede.")
    for i in range(offset, len(available_ips)):
        candidate_ip = available_ips[i]
        if is_ip_valid(candidate_ip):  
            logger.debug(f"IP livre encontrado: {candidate_ip}")
            if configure_ip(candidate_ip):
                return candidate_ip
    raise ValueError("Não foi possível encontrar um IP livre na rede.")

def generate_valid_ips(count):
    try:
        logger.debug(f"Gerando até {count} IPs válidos configurados...")
        available_ips = list_ips()
        valid_ips = []

        for ip in available_ips:
            if len(valid_ips) >= count:
                break
            if is_ip_valid(ip) and configure_ip(ip):
                valid_ips.append(ip)
                logger.debug(f"IP configurado: {ip}")

        if len(valid_ips) < count:
            raise ValueError(f"Não foi possível gerar a quantidade necessária de IPs ({count}).")

        return valid_ips
    except Exception as e:
        logger.error(f"Erro ao gerar IPs válidos configurados: {e}")
        return []



def generate_multicast_group():
    return f"239.{random.randint(0, 255)}.{random.randint(0, 255)}.{random.randint(1, 254)}"




def is_ip_configured(ip: str) -> bool:
    try:
        for _, addrs in net_if_addrs().items():
            for addr in addrs:
                if addr.family == AF_INET and addr.address == ip:
                    return True
        return False
    except Exception as e:
        logger.error(f"Erro ao verificar se o IP {ip} está configurado: {e}")
        return False

def get_network_interface(subnet: str = "192.168.1.") -> str:
    try:
        for iface, addrs in net_if_addrs().items():
            for addr in addrs:
                if addr.family == AF_INET and addr.address.startswith(subnet):
                    return iface
        raise RuntimeError(f"Nenhuma interface encontrada na sub-rede {subnet}")
    except Exception as e:
        logger.error(f"Erro ao buscar a interface de rede para a sub-rede {subnet}: {e}")
        raise

def configure_ip(ip: str) -> None:
    try:
        if is_ip_configured(ip):
            logger.debug(f"O IP {ip} já está configurado.")
            return True

        system = platform.system().lower()

        if system == "linux":
            interface = get_network_interface()
            command = ["sudo", "ip", "addr", "add", f"{ip}/24", "dev", interface]
            subprocess.run(command, check=True)
            logger.debug(f"IP {ip} atribuído com sucesso à interface {interface}.")
            return True

        elif system == "windows":
            interface = get_network_interface()
            command = [
                "netsh", "interface", "ip", "add", "address", interface, ip, "255.255.255.0"
            ]
            subprocess.run(command, check=True)
            logger.debug(f"IP {ip} atribuído com sucesso à interface {interface}.")
            return True

        else:
            raise NotImplementedError(f"Sistema operacional não suportado: {system}")
    except subprocess.CalledProcessError as e:
        logger.error(f"Erro ao executar o comando para atribuir o IP {ip}: {e}")
    except Exception as e:
        logger.error(f"Erro geral ao configurar o IP {ip}: {e}")
        return False

In [35]:
generate_valid_ips(5)

11:28:27 IPCONFIG/2604661836 - Gerando até 5 IPs válidos configurados...
11:28:27 IPCONFIG/2604661836 - Obtendo IP local e máscara de rede...
11:28:27 IPCONFIG/2604661836 - IP local e máscara de rede obtidos: 192.168.1.106, 255.255.255.0
11:28:27 IPCONFIG/2604661836 - Verificando se o IP 192.168.1.1 é válido...
11:28:27 IPCONFIG/2604661836 - Verificando se o IP 192.168.1.2 é válido...
11:28:27 IPCONFIG/2604661836 - Verificando se o IP 192.168.1.3 é válido...
11:28:27 IPCONFIG/2604661836 - Verificando se o IP 192.168.1.4 é válido...
11:28:27 IPCONFIG/2604661836 - Verificando se o IP 192.168.1.5 é válido...
11:28:27 IPCONFIG/2604661836 - Verificando se o IP 192.168.1.6 é válido...
11:28:27 IPCONFIG/2604661836 - Verificando se o IP 192.168.1.7 é válido...
11:28:27 IPCONFIG/2604661836 - Verificando se o IP 192.168.1.8 é válido...
11:28:30 IPCONFIG/2604661836 - IP 192.168.1.8 atribuído com sucesso à interface wlp0s20f3.
11:28:30 IPCONFIG/2604661836 - IP configurado: 192.168.1.8
11:28:30 IPC

['192.168.1.8', '192.168.1.9', '192.168.1.10', '192.168.1.11', '192.168.1.12']

In [22]:
get_local_ip()

'192.168.1.106'

In [36]:
import json
def import_ips_from_json(file_path):
    try:
        with open(file_path, 'r') as json_file:
            ips = json.load(json_file)
        logger.info(f"IPs importados do arquivo JSON: {ips}")
        return ips
    except FileNotFoundError:
        logger.error(f"Arquivo {file_path} não encontrado.")
        return []
    except json.JSONDecodeError as e:
        logger.error(f"Erro ao decodificar o arquivo JSON: {e}")
        return []
    except Exception as e:
        logger.error(f"Erro geral ao importar IPs do arquivo JSON: {e}")
        return []


In [37]:
import_ips_from_json('data/devices_ips.json')

11:48:02 IPCONFIG/1241428177 - IPs importados do arquivo JSON: ['192.168.1.13', '192.168.1.14', '192.168.1.15', '192.168.1.16', '192.168.1.17']


['192.168.1.13',
 '192.168.1.14',
 '192.168.1.15',
 '192.168.1.16',
 '192.168.1.17']